In [ ]:
# from mpl_toolkits.basemap import Basemap
from google.cloud import bigquery
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import numpy as np
import shutil

### Get the closest weather stations in Montreal

In [ ]:
client = bigquery.Client()
stations = client.query("""
    SELECT
          name, id,
          state,
          latitude,
          longitude,
          ST_DISTANCE(
              ST_GEOGPOINT(-73.573570, 45.522490), 
              ST_GEOGPOINT(longitude, latitude)
          ) AS dist_ms 
        FROM
          `bigquery-public-data.ghcn_d.ghcnd_stations`
        ORDER BY
          dist_ms ASC
        LIMIT
          20
    """)
results = stations.result()

In [ ]:
for row in results:
    print("{} : {} : {} : {} : {} : {}".format(row.name, row.id, row.state, row.latitude, row.longitude, row.dist_ms))

### Get the weather features from the weather stations

In [ ]:
def get_weather(year):
    weather = client.query("""
            SELECT
              date,
              IF (element = 'PRCP', value/10, NULL) AS prcp,
              IF (element = 'TMIN', value/10, NULL) AS tmin,
              IF (element = 'TMAX', value/10, NULL) AS tmax,
              IF (element = 'SNOW', value, NULL) AS snow,
              IF (element = 'SNWD', value / 10, NULL) AS snwd,
              IF (element = 'ACMD', value, NULL) AS acmh,
              IF (element = 'ACSH', value, NULL) AS acsh,
              IF (element = 'AWDR', value, NULL) AS awdr,
              IF (element = 'AWND', value, NULL) AS awnd,
              IF (element = 'FRGB', value, NULL) AS frgb,
              IF (element = 'FRGT', value, NULL) AS frgt,
              IF (element = 'FRTH', value, NULL) AS frth,
              IF (element = 'EVAP', value, NULL) AS evap,
              IF (element = 'PSUN', value, NULL) AS psun,
              IF (SUBSTR(element, 0, 2) = 'SN', value / 10, NULL) AS sntmp,
              IF (SUBSTR(element, 0, 2) = 'SX', value / 10, NULL) AS sxtmp,
              IF (element = 'TAVG', value / 10, NULL) AS tavg,
              IF (element = 'THIC', value / 10, NULL) AS thic,
              IF (element = 'TSUN', value, NULL) AS tsun,
              IF (element = 'WDFG', value, NULL) AS wdfg,
              IF (element = 'WDFI', value, NULL) AS wdfi,
              IF (element = 'WDFM', value, NULL) AS wdfm,
              IF (element = 'WDMV', value, NULL) AS wdmv,
              IF (element = 'WESD', value / 10, NULL) AS wesd,
              IF (element = 'WESF', value / 10, NULL) AS wesf,
              IF (element = 'WSFG', value / 10, NULL) AS wsfg,
              IF (element = 'WT01', 1, 0) AS wt_fog,
              IF (element = 'WT02', 1, 0) AS wt_heavy_fog,
              IF (element = 'WT03', 1, 0) AS wt_thunder,
              IF (element = 'WT04', 1, 0) AS wt_pellet,
              IF (element = 'WT05', 1, 0) AS wt_hail,
              IF (element = 'WT06', 1, 0) AS wt_glaze,
              IF (element = 'WT07', 1, 0) AS wt_dust,
              IF (element = 'WT08', 1, 0) AS wt_smoke,
              IF (element = 'WT09', 1, 0) AS wt_blow,
              IF (element = 'WT10', 1, 0) AS wt_tornado,
              IF (element = 'WT11', 1, 0) AS wt_dam_wind,
              IF (element = 'WT12', 1, 0) AS wt_spray,
              IF (element = 'WT13', 1, 0) AS wt_mist,
              IF (element = 'WT14', 1, 0) AS wt_drizzle,
              IF (element = 'WT15', 1, 0) AS wt_freez_drizzle,
              IF (element = 'WT16', 1, 0) AS wt_rain,
              IF (element = 'WT17', 1, 0) AS wt_freeze_rain,
              IF (element = 'WT18', 1, 0) AS wt_ice,
              IF (element = 'WT19', 1, 0) AS wt_unk_prpt,
              IF (element = 'WT20', 1, 0) AS wt_gr_fog,
              IF (element = 'WT21', 1, 0) AS wt_ice_fog
          FROM
            `bigquery-public-data.ghcn_d.ghcnd_{}`
          WHERE
            id = 'CA007024745'
            AND qflag IS NULL
          ORDER BY 
            date ASC
        """.format(year))
    results = weather.result()
    return results

In [ ]:
cols = ["date", "prcp", "tmin", "tmax", "snow", "snwd", "acmh", "acsh", "awdr", "awnd", "frgb", "frgt", "frth",
        "evap", "psun", "sntmp", "sxtmp", "tavg", "thic", "tsun", "wdfg", "wdfi", "wdfm", "wdmv", 
        "wesd", "wsfg", "wt_fog", "wt_heavy_fog", "wt_thunder", "wt_pellet", "wt_hail", "wt_glaze",
        "wt_dust", "wt_smoke", "wt_blow", "wt_tornado", "wt_dam_wind", "wt_spray", "wt_mist", "wt_drizzle",
        "wt_freez_drizzle", "wt_rain", "wt_freeze_rain", "wt_ice", "wt_unk_prpt", "wt_gr_fog", "wt_ice_fog"]

data = {v : [] for v in cols}

for y in [2018, 2019, 2020]:
    res = get_weather(y)
    for r in res:
        for c in cols:
            data[c].append(getattr(r, c))

        
df = pd.DataFrame(data=data)
df.head()

In [ ]:
num_cols = [
    'prcp', 'tmin', 'tmax', 'snwd', 'sntmp', 'tavg', 'wdfg', 'wsfg'
]
bool_cols = [
   'wt_fog', 'wt_heavy_fog', 'wt_thunder', 'wt_pellet', 'wt_hail',
   'wt_glaze', 'wt_dust', 'wt_smoke', 'wt_blow', 'wt_tornado',
   'wt_dam_wind', 'wt_spray', 'wt_mist', 'wt_drizzle', 'wt_freez_drizzle',
   'wt_rain', 'wt_freeze_rain', 'wt_ice', 'wt_unk_prpt', 'wt_gr_fog',
   'wt_ice_fog'
]


groupby_dict = {}
groupby_dict.update({nc : ['min', 'max', 'mean'] for nc in num_cols})
groupby_dict.update({bc : ['max'] for bc in bool_cols})
new_cols = ["date"] + [agg + "_" + nc for nc in num_cols for agg in ['min', 'max', 'mean']] + bool_cols

out = df.dropna(axis=1, how='all')
out = out.groupby("date").agg(groupby_dict).reset_index()
out.columns = out.columns.droplevel()
out.columns = new_cols
out.head()

In [ ]:
def getDuplicateColumns(df):
    '''
    Get a list of duplicate columns.
    It will iterate over all the columns in dataframe and find the columns whose contents are duplicate.
    :param df: Dataframe object
    :return: List of columns whose contents are duplicates.
    '''
    duplicateColumnNames = set()
    # Iterate over all the columns in dataframe
    for x in range(df.shape[1]):
        # Select column at xth index.
        col = df.iloc[:, x]
        # Iterate over all the columns in DataFrame from (x+1)th index till end
        for y in range(x + 1, df.shape[1]):
            # Select column at yth index.
            otherCol = df.iloc[:, y]
            # Check if two columns at x 7 y index are equal
            if col.equals(otherCol):
                duplicateColumnNames.add(df.columns.values[y])
 
    return list(duplicateColumnNames)

- Time period: 2020, don't have 2017
- prcp: rain precipitation in millimeters
- tmin: minimum temperature in degree celsius
- tmax: maximum temperature in degree celsius
- snwd: snow depth (mm)
- tavg: average temperature (degree celsius)
- wdfg: direction of peak wind gust (degree)
- wsfg: peak gust wind speed (m / s)

In [ ]:
dup_cols = getDuplicateColumns(out) + ["wt_fog"]
out = out.drop(columns=dup_cols)
out.head()

In [ ]:
gs_uri = 'gs://videotron-ai-bucket/'
dataset_path = gs_uri + 'dataset/'
df.to_csv(dataset_path + 'daily_weather_montreal.csv', index=False)